In [ ]:
`from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import glob
import json
import cv2
import copy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json
import cv2
import copy

from tqdm.autonotebook import tqdm
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

<ipython-input-3-e1e031804310>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
def get_video_files(path):
  video_files =  glob.glob(path+'*.mp4')

  # print(video_files)

  frame_count = []
  for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
      video_files.remove(video_file)
      continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
  print("frames" , frame_count)
  print("Total number of videos: " , len(frame_count))
  print('Average frame per video:',np.mean(frame_count))

  return video_files

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=9d9247280278126ef042a2d297a42c25e73da5e024ff4f258cc87f7d65ed94cf
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import dlib

print("Is dlib compiled with CUDA:", dlib.DLIB_USE_CUDA)
print("CUDA Device Count:", dlib.cuda.get_num_devices())

Is dlib compiled with CUDA: True
CUDA Device Count: 1


In [ ]:
import face_recognition
# face_recognition.face_distance([], [], use_gpu=False)

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

In [ ]:
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    flag = 0
    frames = []
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
# Celeb_DF
# video_files = get_video_files('/content/drive/MyDrive/Datasets/Celeb_DF/FAKE')
# create_face_videos(video_files,'/content/drive/MyDrive/Datasets/Celeb_DF/FAKE_face_only')

# video_files = get_video_files('/content/drive/MyDrive/Datasets/Celeb_DF/REAL')
# create_face_videos(video_files,'/content/drive/MyDrive/Datasets/Celeb_DF/REAL_face_only')

In [ ]:
# !mkdir '/content/drive/My Drive/Datasets/REAL_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_data'


# !mkdir '/content/drive/My Drive/Datasets/Real_Face_Dataset'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset'
# !mkdir '/content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/NeuralTextures_Fake_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/FaceSwap_Fake_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/FaceShifter_Fake_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/Face2Face_Fake_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/DeepFakeDetection_Fake_Face_data'
# !mkdir '/content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data'

In [ ]:
# youtube real data
video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/')
create_face_videos(video_files,'/content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/')

# actors real data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/')

# # NeuralTextures Fake data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/NeuralTextures/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/NeuralTextures_Fake_Face_data/')

# # FaceSwap Fake data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/FaceSwap/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/FaceSwap_Fake_Face_data/')

# # FaceShifter Fake data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/FaceShifter/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/FaceShifter_Fake_Face_data/')

# # Face2Face Fake data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Face2Face/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/Face2Face_Fake_Face_data/')

# # Deepfakes Fake data 999
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/')

# # DeepFakeDetection Fake data
# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/')
# create_face_videos(video_files,'/content/drive/My Drive/Datasets/Fake_Face_Dataset/DeepFakeDetection_Fake_Face_data/')


In [ ]:
# # youtube real data
# # #to Save the result of "get_video_files" function in a json file

# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/')
# json_file_path = '/content/drive/MyDrive/getVedFiles/youtube.json'
# with open(json_file_path, 'w') as file:
#     json.dump(video_files, file)

In [ ]:
# Load the JSON file that contains videos paths

json_file_path = '/content/drive/MyDrive/getVedFiles/youtube.json'
with open(json_file_path, 'r') as file:
    youtube_video_files = json.load(file)

print(youtube_video_files)  # Verify the loaded list

['/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/585.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/599.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/469.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/481.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/183.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/253.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/672.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/720.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/866.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/878.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/youtube/c23/videos/339.mp4', '/content

In [ ]:
create_face_videos(youtube_video_files,'/content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/')

No of videos already present  1000


  0%|          | 0/1000 [00:00<?, ?it/s]

File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/585.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/599.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/469.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/481.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/183.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/253.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/672.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/720.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/youtube_Real_Face_data/866.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face

In [ ]:
# # actors real data
# # #to Save the result of "get_video_files" function in a json file

# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/')
# json_file_path = '/content/drive/MyDrive/getVedFiles/actors.json'
# with open(json_file_path, 'w') as file:
#     json.dump(video_files, file)

In [ ]:
# Load the JSON file that contains videos paths

json_file_path = '/content/drive/MyDrive/getVedFiles/actors.json'
with open(json_file_path, 'r') as file:
    actors_video_files = json.load(file)

print(actors_video_files)  # Verify the loaded list

['/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/07__talking_against_wall.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/09__walking_down_street_outside_angry.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/13__talking_against_wall.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/13__walking_down_indoor_hall_disgust.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/18__kitchen_pan.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/27__kitchen_pan.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/06__outside_talking_still_laughing.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/01__kitchen_pan.mp4', '/content/drive/MyDrive/FaceForensicsData/original_sequences/actors/c23/videos/08__podium_speech_happy.mp4', '/

In [ ]:
create_face_videos(actors_video_files,'/content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/')

No of videos already present  363


  0%|          | 0/363 [00:00<?, ?it/s]

File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/07__talking_against_wall.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/09__walking_down_street_outside_angry.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/13__talking_against_wall.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/13__walking_down_indoor_hall_disgust.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/18__kitchen_pan.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/27__kitchen_pan.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/06__outside_talking_still_laughing.mp4
File Already exists:  /content/drive/My Drive/Datasets/Real_Face_Dataset/actors_real_Face_data/01__kitchen_pan.mp4


In [ ]:
# #Deepfakes
# #to Save the result of "get_video_files" function in a json file

# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/')
# json_file_path = '/content/drive/MyDrive/getVedFiles/Deepfakes.json'
# with open(json_file_path, 'w') as file:
#     json.dump(video_files, file)

In [ ]:
# Load the JSON file that contains videos paths

json_file_path = '/content/drive/MyDrive/getVedFiles/Deepfakes.json'
with open(json_file_path, 'r') as file:
    deepfakes_video_files = json.load(file)

print(deepfakes_video_files)  # Verify the loaded list

['/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/599_585.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/469_481.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/481_469.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/183_253.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/253_183.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/672_720.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/720_672.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/866_878.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/878_866.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/Deepfakes/c23/videos/339_392.mp4', '/content

In [ ]:
# Now use the loaded list with create_face_videos function
create_face_videos(deepfakes_video_files, '/content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/')

No of videos already present  999


  0%|          | 0/999 [00:00<?, ?it/s]

File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/599_585.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/469_481.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/481_469.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/183_253.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/253_183.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/672_720.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/720_672.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/866_878.mp4
File Already exists:  /content/drive/My Drive/Datasets/Fake_Face_Dataset/Deepfakes_Fake_Face_data/878_866.mp4
File Alrea

In [ ]:
# # DeepFakeDetection
# # to Save the result of "get_video_files" function in a json file

# video_files = get_video_files('/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/')

# json_file_path = '/content/drive/MyDrive/getVedFiles/DeepFakeDetection.json'
# with open(json_file_path, 'w') as file:
#     json.dump(video_files, file)

In [ ]:
# Load the list from the JSON f# Load the JSON file that contains videos paths
json_file_path = '/content/drive/MyDrive/getVedFiles/DeepFakeDetection.json'
with open(json_file_path, 'r') as file:
    DeepFakeDetection_video_files = json.load(file)

print(DeepFakeDetection_video_files)  # Verify the loaded list

['/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/20_09__talking_angry_couch__0IRM5ADD.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/10_19__talking_angry_couch__QNZIMEJH.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/02_14__podium_speech_happy__3IUBEKCT.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/02_06__secret_conversation__O9WOC1KJ.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/26_04__talking_angry_couch__MS2MLJP1.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/02_09__secret_conversation__6KUOFMZW.mp4', '/content/drive/MyDrive/FaceForensicsData/manipulated_sequences/DeepFakeDetection/c23/videos/04_26__talking_angry_couch__WX836VLY.mp4', '/content/drive/MyDrive/FaceForensicsData/manip

In [ ]:
# Now use the loaded list with create_face_videos function
# create_face_videos(DeepFakeDetection_video_files, '/content/drive/My Drive/Datasets/Fake_Face_Dataset/DeepFakeDetection_Fake_Face_data/')
